In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Food Recognition Project

/content/drive/MyDrive/Food Recognition Project


In [ ]:
!ls

 background_removal.ipynb   CLIP_FM.ipynb   New_Output	 Output  'Resized Dataset'


In [ ]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install torch torchvision transformers

In [ ]:
import os
import clip
import torch
from PIL import Image
import pandas as pd
import random
import glob
import numpy as np
from tqdm import tqdm

device = "cpu"
model, preprocess = clip.load('ViT-B/32', device)



In [ ]:
from multiprocessing import Pool

In [ ]:
import multiprocessing
import os
pool_count = multiprocessing.cpu_count()
print(f'The number of pools available is: {pool_count}')

The number of pools available is: 2


In [ ]:
#Creating Dataframes

In [ ]:
class_names = os.listdir('./Resized Dataset/')
class_names

['commercial_air_normal',
 'commercial_air_over',
 'commercial_deep_normal',
 'commercial_deep_over',
 'commercial_unbaked',
 'inhouse_air_normal',
 'inhouse_air_over',
 'inhouse_deep_normal',
 'inhouse_deep_over',
 'inhouse_old_air_normal',
 'inhouse_old_air_over',
 'inhouse_old_deep_normal',
 'inhouse_old_deep_over',
 'inhouse_unbaked']

In [ ]:

# Define the root directory of your dataset
dataset_root = 'Resized Dataset'

# Create an empty dictionary to store dataframes for each class
class_dataframes = {}


class_folders = sorted(os.listdir(dataset_root))
for class_folder in class_folders:
    class_path = os.path.join(dataset_root, class_folder)

    class_data = []

    # Loop through images in each class folder
    # Select random 200 images from the class
    rand200_images = random.sample(os.listdir(class_path), 200)
    for image_file in rand200_images:
        image_path = os.path.join(class_path, image_file)


        class_data.append({
            'ImagePath': image_path,
            'Class': class_folder,
            'Predicted Class R1': None,
            'Predicted Class R3': None,
            'Predicted Class R5': None,
        })

    class_df = pd.DataFrame(class_data)

    class_dataframes[class_folder] = class_df

print(class_dataframes['commercial_air_normal'])


In [ ]:
print(class_dataframes['commercial_air_normal'])

In [ ]:
candidate_captions = ['A picture of ' + class_name for class_name in class_names]
print(candidate_captions)

In [ ]:
# Test
import glob


def argmax(iterable):
  return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

correct = []

text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
  class_correct = []
  test_imgs = glob.glob('./Test/' + cls + '/*.jpg')
  for img in test_imgs:
    image = preprocess(Image.open(img)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image,text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        print(probs)
        pred = class_names[argmax(list(probs)[0])]

        if pred == cls:
          correct.append(1)
          class_correct.append(1)
        else:
          correct.append(0)
          class_correct.append(0)

  print('Accuracy on class ' + cls + ' is: ' + str(sum(class_correct)/len(class_correct)))
print('Accuracy on all is: ' + str(sum(correct)/len(correct)))

In [ ]:
def top_n_classes(iterable, n, class_names):
    # Get the indices of the top n classes as per the probability
    top_indices = sorted(range(len(iterable[0])), key=lambda i: iterable[0][i], reverse=True)[:n]

    # Get the corresponding class names
    top_class_names = [class_names[i] for i in top_indices]

    return top_class_names


In [ ]:

def runCLIP(cls, candidate_captions, class_dataframes, class_names):

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model, preprocess = clip.load("ViT-B/32", device=device)


  text = clip.tokenize(candidate_captions).to(device)

  for index, row in class_dataframes[cls].iterrows():
    curr_image_path = row["ImagePath"]
    image = preprocess(Image.open(curr_image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image,text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()


        # Adding the predicted class to dataframe R1
        predR1 = top_n_classes(probs, 1, class_names)
        class_dataframes[cls].at[index, 'Predicted Class R1'] = predR1

        # Adding the predicted class to dataframe R3
        predR3 = top_n_classes(probs, 3, class_names)
        class_dataframes[cls].at[index, 'Predicted Class R3'] = predR3

        # Adding the predicted class to dataframe R5
        predR5 = top_n_classes(probs, 5, class_names)
        class_dataframes[cls].at[index, 'Predicted Class R5'] = predR5


In [ ]:
# Calculating R1 Accuracy
def R1_Accuracy(class_names, class_dataframes):
  for cls in class_names:
    curr_df =  class_dataframes[cls]


    curr_df['R1 Matches'] = curr_df.apply(lambda row: row['Predicted Class R1'][0] == row['Class'], axis=1)
    correct_predictions = curr_df['R1 Matches'].sum()
    total_predictions = len(curr_df)
    accuracy = (correct_predictions / total_predictions) * 100

    print(f"The accuracy of {cls} is {accuracy:f}%")

In [ ]:
# Calculating R3 Accuracy
def R3_Accuracy(class_names, class_dataframes):
  for cls in class_names:
      curr_df = class_dataframes[cls]

      curr_df['R3 Matches'] = curr_df.apply(lambda row: any(class_value == row['Class'] for class_value in row['Predicted Class R3']), axis=1)
      correct_predictions = curr_df['R3 Matches'].sum()
      total_predictions = len(curr_df)
      accuracy = (correct_predictions / total_predictions) * 100

      print(f"The accuracy of {cls} is {accuracy:f}%")


In [ ]:
# Calculating R5 Accuracy
def R5_Accuracy(class_names, class_dataframes):
  for cls in class_names:
      curr_df = class_dataframes[cls]

      curr_df['R5 Matches'] = curr_df.apply(lambda row: any(class_value == row['Class'] for class_value in row['Predicted Class R5']), axis=1)
      correct_predictions = curr_df['R5 Matches'].sum()
      total_predictions = len(curr_df)
      accuracy = (correct_predictions / total_predictions) * 100

      print(f"The accuracy of {cls} is {accuracy:f}%")

In [ ]:
# Saving the Reultant Dataframes as csv
def save(output_directory, class_dataframes):
  # output_directory = './Output/'

  for class_name, df in class_dataframes.items():

      output_file_path = f"{output_directory}{class_name}.csv"

      # Save the DataFrame to a CSV file
      df.to_csv(output_file_path, index=False)

In [ ]:
# Running CLIP for all classes
for cls in tqdm(class_names):
  runCLIP(cls, candidate_captions, class_dataframes, class_names)

R1_Accuracy(class_names, class_dataframes)
R3_Accuracy(class_names, class_dataframes)
R5_Accuracy(class_names, class_dataframes)
save('./Output/', class_dataframes)


# Removing Redundant classes and Running Clip on them (Final 8 classes)

In [ ]:
new_class_names = ['commercial_deep_normal',
 'commercial_deep_over',
 'commercial_unbaked',
 'inhouse_deep_normal',
 'inhouse_deep_over',
 'inhouse_old_deep_normal',
 'inhouse_old_deep_over',
 'inhouse_unbaked']

In [ ]:
new_candidate_captions = ['A picture of ' + class_name for class_name in new_class_names]
print(new_candidate_captions)

['A picture of commercial_deep_normal', 'A picture of commercial_deep_over', 'A picture of commercial_unbaked', 'A picture of inhouse_deep_normal', 'A picture of inhouse_deep_over', 'A picture of inhouse_old_deep_normal', 'A picture of inhouse_old_deep_over', 'A picture of inhouse_unbaked']


In [ ]:
# Define the root directory of your dataset
dataset_root = 'Resized Dataset'

# Create an empty dictionary to store dataframes for each class
new_class_dataframes = {}

class_folders = sorted(os.listdir(dataset_root))
new_class_folders = [folder for folder in class_folders if folder in new_class_names]

for class_folder in new_class_folders:
    class_path = os.path.join(dataset_root, class_folder)

    class_data = []

    # Loop through images in each class folder
    # Select random 200 images from the class
    rand200_images = random.sample(os.listdir(class_path), 200)
    for image_file in rand200_images:
        image_path = os.path.join(class_path, image_file)


        class_data.append({
            'ImagePath': image_path,
            'Class': class_folder,
            'Predicted Class R1': None,
            'Predicted Class R3': None,
            'Predicted Class R5': None,
        })

    class_df = pd.DataFrame(class_data)

    new_class_dataframes[class_folder] = class_df

print(new_class_dataframes['commercial_deep_normal'])


In [ ]:
for cls in tqdm(new_class_names):
  runCLIP(cls, new_candidate_captions, new_class_dataframes, new_class_names)

100%|██████████| 8/8 [1:16:36<00:00, 574.52s/it]


In [ ]:
R1_Accuracy(new_class_names, new_class_dataframes)

The accuracy of commercial_deep_normal is 0.000000%
The accuracy of commercial_deep_over is 0.000000%
The accuracy of commercial_unbaked is 98.500000%
The accuracy of inhouse_deep_normal is 0.000000%
The accuracy of inhouse_deep_over is 0.000000%
The accuracy of inhouse_old_deep_normal is 0.000000%
The accuracy of inhouse_old_deep_over is 0.000000%
The accuracy of inhouse_unbaked is 40.000000%


In [ ]:
R3_Accuracy(new_class_names, new_class_dataframes)

The accuracy of commercial_deep_normal is 3.000000%
The accuracy of commercial_deep_over is 71.500000%
The accuracy of commercial_unbaked is 100.000000%
The accuracy of inhouse_deep_normal is 0.500000%
The accuracy of inhouse_deep_over is 18.000000%
The accuracy of inhouse_old_deep_normal is 2.000000%
The accuracy of inhouse_old_deep_over is 14.500000%
The accuracy of inhouse_unbaked is 100.000000%


In [ ]:
R5_Accuracy(new_class_names, new_class_dataframes)

The accuracy of commercial_deep_normal is 89.500000%
The accuracy of commercial_deep_over is 88.500000%
The accuracy of commercial_unbaked is 100.000000%
The accuracy of inhouse_deep_normal is 15.000000%
The accuracy of inhouse_deep_over is 86.000000%
The accuracy of inhouse_old_deep_normal is 18.500000%
The accuracy of inhouse_old_deep_over is 53.000000%
The accuracy of inhouse_unbaked is 100.000000%


In [ ]:
save('./New_Output/', new_class_dataframes)

# Analysis of the remaining 6 classes


In [ ]:
remaining_class_names =['commercial_air_normal',
 'commercial_air_over',
 'inhouse_air_normal',
 'inhouse_air_over',
 'inhouse_old_air_normal',
 'inhouse_old_air_over']

In [ ]:
# Define the root directory of your dataset
dataset_root = 'Resized Dataset'

# Create an empty dictionary to store dataframes for each class
remaining_class_dataframes = {}

class_folders = sorted(os.listdir(dataset_root))
remaining_class_folders = [folder for folder in class_folders if folder in remaining_class_names]

for class_folder in remaining_class_folders:
    class_path = os.path.join(dataset_root, class_folder)

    class_data = []

    # Loop through images in each class folder
    # Select random 200 images from the class
    rand200_images = random.sample(os.listdir(class_path), 200)
    for image_file in rand200_images:
        image_path = os.path.join(class_path, image_file)


        class_data.append({
            'ImagePath': image_path,
            'Class': class_folder,
            'Predicted Class R1': None,
            'Predicted Class R3': None,
            'Predicted Class R5': None,
        })

    class_df = pd.DataFrame(class_data)

    remaining_class_dataframes[class_folder] = class_df

print(remaining_class_dataframes['commercial_air_normal'])


In [ ]:
for cls in tqdm(remaining_class_names):
  runCLIP(cls, new_candidate_captions, remaining_class_dataframes, new_class_names)

100%|██████████| 6/6 [1:02:23<00:00, 623.90s/it]


In [ ]:
save('./Remaining_Output/', remaining_class_dataframes)